In [1]:
import pandas as pd
import numpy as np

In [2]:
aulas = pd.read_excel('/home/cayena/Downloads/planilha_teste_thefamily.xlsx')

In [3]:
aulas['Horário'] = aulas['Horário'].astype(str)

In [4]:
aulas['segunda'] = aulas['Dias da Semana'].str.contains('2ª').astype(int)
aulas['terça'] = aulas['Dias da Semana'].str.contains('3ª').astype(int)
aulas['quarta'] = aulas['Dias da Semana'].str.contains('4ª').astype(int)
aulas['quinta'] = aulas['Dias da Semana'].str.contains('5ª').astype(int)

In [5]:
df_horarios = aulas['Horário'].str.get_dummies()
df_unidade = aulas['Unidade'].str.get_dummies()

In [6]:
aulas_geral = pd.concat([aulas, df_horarios,df_unidade], axis=1)

In [16]:
grupo_semana = aulas_geral[['Grupo', 'segunda', 'terça', 'quarta', 'quinta']].copy()
grupo_horario = aulas_geral.drop(columns=[ 'Horário', 'Unidade', 'Dias da Semana','segunda', 'terça', 'quarta', 'quinta', 'Jardim', 'Satélite', 'Vicentina']).copy()
grupo_loc = aulas_geral[['Grupo', 'Jardim','Satélite','Vicentina']].copy()

In [21]:
Professores = {'Name':['Allan','Carlos','Henrique','Yasmin','Malu','Tanze','Okamoto']}
profes = pd.DataFrame(Professores)
profes.head()

,Name
0,Allan
1,Carlos
2,Henrique
3,Yasmin
4,Malu


In [18]:
grupo_loc

,Grupo,Jardim,Satélite,Vicentina
0,MBA DAVI BRAGA,0,0,1
1,MBA LIGA DOS CAMPEÕES,0,0,1
2,MBA IDEIAS EM AÇÃO,0,0,1
3,SAVANAH PRESENCIAL,0,0,1
4,CONVERSATION 14 PRESENCIAL,0,0,1
...,...,...,...,...
71,FLAGSTAFF PRESENCIAL,0,1,0
72,BERMUDAS PRESENCIAL,0,1,0
73,AUCKLAND PRESENCIAL,0,1,0
74,QUEBEC PRESENCIAL,0,1,0
